In [ ]:
import dcarte
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec

import pyarrow as pa
import pyarrow.parquet as pq

import seaborn as sns
pd.set_option('display.max_columns', None)

In [ ]:
ids = np.load('ids_seven_days_each_five_bimonth.npy')
len(ids)

In [ ]:
patient = dcarte.load('Patients','lookup',reload=False)
patient['participant_id'] = patient['id'].apply(lambda x:x[:5])
patient = patient[patient['participant_id'].isin(ids)]
#patient

### I have 8 without birthdats in DCARTE

In [ ]:
patient[patient.isna().any(axis=1)]

## Exploring dyads

In [ ]:
dyads = pd.read_excel('dyads_unprotected.xlsx')
print('In total ',len(dyads))

In [ ]:
dyads['participant_id'] = dyads['Research ID'].apply(lambda x:x[:5])
dyads = dyads[dyads['participant_id'].isin(ids)]
dyads

### I have one without DYAD

In [ ]:
missing = list(sorted(set(ids) - set(dyads['participant_id'].unique())))
print(missing)

In [ ]:
patient[patient['participant_id']=='R8amT']

In [ ]:
#demographics = pd.read_csv('MinderHealthManageme-Demographics_DATA_LABELS_2023-10-06_1207.csv')306
demographics = pd.read_csv('redcap_16_nov_2023/Demographics_MinderHealthManageme_DATA_LABELS_2023-11-16_1127.csv')
len(demographics.columns.to_list())

In [ ]:
demographics = demographics[['Dyad number', 'Year of birth PwD', 'Gender PwD',
 'Employment - PwD','Education - PwD','Does PwD smoke?',
 'Does PwD drink alcohol?', 'House style', 'Mobility','Symptoms onset year', 'Does PwD live alone?', 'Cohort site']]
#diagnosis, imaging results and regions affected, comorbidities, smoke, drink, house style, mobility, pets
demographics  = demographics.rename(columns={"Dyad number": "DYAD"})
demographics.head(3)

In [ ]:
merge1 = dyads.merge(demographics, on='DYAD')
merge1

In [ ]:
merge1['Cohort site'].unique()

merge1[merge1['Cohort site'].isna()]

In [ ]:
merge2 = merge1.merge(patient, on='participant_id',how='right')
merge2[merge2['Year of birth PwD']!=merge2['birth_date']]

In [ ]:
merge2['birth_date'].fillna(merge2['Year of birth PwD'], inplace=True)

In [ ]:
merge2 = merge2[['participant_id', 'DYAD','birth_date', 'gender', 'Employment - PwD', 'Education - PwD', 'Does PwD smoke?',
       'Does PwD drink alcohol?', 'House style', 'Mobility',
       'Symptoms onset year', 'Does PwD live alone?', 'Cohort site',
       'organization']]

merge2[merge2.isna().any(axis=1)]

In [ ]:
merge2.to_excel("redcap_demographics_nov16.xlsx")